# 2024_02_05: code updated and simulations for Y = f(X) coverage probability is run

## Preliminaries

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import scipy.stats as st
import scipy.special as sp
import random

from fastkde import fastKDE

seed_val = 1234

In [2]:
def estimator(x, alpha = 0.05):
    
    if(x.shape[0]%2!=0):
        x = x[:-1]
    
    estim, inf = np.split(x, 2) #split data up into two halves
    
    ## first split used for density estimation
    margin_x = fastKDE.pdf_at_points(var1 = estim[:,0], list_of_points = list(inf[:,0]))
    margin_y = fastKDE.pdf_at_points(var1 = estim[:,1], list_of_points = list(inf[:,1]))
    select = np.logical_and(margin_x > 0, margin_y > 0)
    margin_y = margin_y[select]
    margin_x = margin_x[select]
    
    h_x1 = -np.mean(np.log(margin_x))
    h_y1 = -np.mean(np.log(margin_y))
    covar1 = np.cov(np.log(margin_x), np.log(margin_y))
    delta_var1 = covar1[0,0] + covar1[1,1] - 2*covar1[0,1]
    
    
    ## second split used for density estimation
    margin_x = fastKDE.pdf_at_points(var1 = inf[:,0], list_of_points = list(estim[:,0]))
    margin_y = fastKDE.pdf_at_points(var1 = inf[:,1], list_of_points = list(estim[:,1]))
    select = np.logical_and(margin_x > 0, margin_y > 0)
    margin_y = margin_y[select]
    margin_x = margin_x[select]
    
    h_x2 = -np.mean(np.log(margin_x))
    h_y2 = -np.mean(np.log(margin_y))
    covar2 = np.cov(np.log(margin_x), np.log(margin_y))
    delta_var2 = covar2[0,0] + covar2[1,1] - 2*covar2[0,1]
    
    ## cross fitting
    h_x = (h_x1 + h_x2)/2
    h_y = (h_y1 + h_y2)/2
    delta = (h_x - h_y)
    
    ## variance estimation using monte carlo
    delta_var = (delta_var1 + delta_var2)/2
    delta_sd = np.sqrt(delta_var)
    
    delta_lcb = delta - st.norm.ppf(1 - alpha/2)*delta_sd/np.sqrt(len(select))
    delta_ucb = delta + st.norm.ppf(1 - alpha/2)*delta_sd/np.sqrt(len(select))
    
    return ([h_x, h_y, delta_lcb, delta, delta_ucb, delta_sd/np.sqrt(len(select))])

## Case 1: $X \sim $ lognormal; $Y \sim $ normal (vary sample size n)

In [3]:
np.random.seed(4321)
def data_gen_1(n = 1000, mu = 5):
    y = np.random.normal(loc = mu, scale = 1, size = n) 
    x = np.random.lognormal(mean = mu, sigma = 1, size = n) 
    return np.column_stack((x, y))

niter = 200
count = np.zeros(niter)
bias = np.zeros(niter)
estim = np.zeros(niter)
sd_estim = np.zeros(niter)
for i in range(niter):
    data = data_gen_1(250, 5)
    op = estimator(data)
    estim[i] = op[3]
    sd_estim[i] = op[5]
    if((5 - op[2])*(op[4] - 5) >= 0):
        count[i] = 1
    bias[i] = np.abs(op[3] - 5)

In [4]:
print(np.round(np.mean(count), 3)) ## coverage
print(np.round([np.mean(bias)], 3)) ## absolute error and error variance
print(np.round([np.mean(estim), np.std(estim)], 3))## MC-based estimator mean and variance
print(np.round(np.mean(sd_estim), 3))

0.945
[0.095]
[5.036 0.112]
0.114


In [5]:
np.random.seed(4321)
def data_gen_1(n = 1000, mu = 5):
    y = np.random.normal(loc = mu, scale = 1, size = n) 
    x = np.random.lognormal(mean = mu, sigma = 1, size = n) 
    return np.column_stack((x, y))

niter = 200
count = np.zeros(niter)
bias = np.zeros(niter)
estim = np.zeros(niter)
sd_estim = np.zeros(niter)
for i in range(niter):
    data = data_gen_1(500, 5)
    op = estimator(data)
    estim[i] = op[3]
    sd_estim[i] = op[5]
    if((5 - op[2])*(op[4] - 5) >= 0):
        count[i] = 1
    bias[i] = np.abs(op[3] - 5)

In [6]:
print(np.round(np.mean(count), 3)) ## coverage
print(np.round([np.mean(bias)], 3)) ## absolute error and error variance
print(np.round([np.mean(estim), np.std(estim)], 3))## MC-based estimator mean and variance
print(np.round(np.mean(sd_estim), 3))

0.98
[0.058]
[5.015 0.073]
0.084


In [7]:
np.random.seed(4321)
def data_gen_1(n = 1000, mu = 5):
    y = np.random.normal(loc = mu, scale = 1, size = n) 
    x = np.random.lognormal(mean = mu, sigma = 1, size = n) 
    return np.column_stack((x, y))

niter = 200
count = np.zeros(niter)
bias = np.zeros(niter)
estim = np.zeros(niter)
sd_estim = np.zeros(niter)
for i in range(niter):
    data = data_gen_1(750, 5)
    op = estimator(data)
    estim[i] = op[3]
    sd_estim[i] = op[5]
    if((5 - op[2])*(op[4] - 5) >= 0):
        count[i] = 1
    bias[i] = np.abs(op[3] - 5)

In [8]:
print(np.round(np.mean(count), 3)) ## coverage
print(np.round([np.mean(bias)], 3)) ## absolute error and error variance
print(np.round([np.mean(estim), np.std(estim)], 3))## MC-based estimator mean and variance
print(np.round(np.mean(sd_estim), 3))

0.965
[0.048]
[5.012 0.06 ]
0.07


## Case 2: $X \sim$ Exp(scale = 1); $Y \sim$ Weibull(scale = 1, shape = 1.5) (vary sample size n)

In [9]:
np.random.seed(1234)
def data_gen_2(n = 1000, scale = 1, shape = 2):
    x = np.random.exponential(scale = 1, size = n)
    y = np.random.weibull(a = shape, size = n)
    return np.column_stack((x, y))

scale = 1
shape = 1.5
true = (1 + np.log(scale)) - (-sp.digamma(1)*(1 - 1/shape) + np.log(scale/shape) + 1) 
niter = 200
count = np.zeros(niter)
bias = np.zeros(niter)
estim = np.zeros(niter)
sd_estim = np.zeros(niter)
for i in range(niter):
    data = data_gen_2(250, scale, shape)
    op = estimator(data)
    estim[i] = op[3]
    sd_estim[i] = op[5]
    if((true - op[2])*(op[4] - true) >= 0):
        count[i] = 1
    bias[i] = np.abs(op[3] - true)

In [10]:
print(np.round(np.mean(count), 3)) ## coverage
print(np.round([np.mean(bias)], 3)) ## absolute error and error variance
print(np.round([np.mean(estim), np.std(estim)], 3))## MC-based estimator mean and variance
print(np.round(np.mean(sd_estim), 3))

0.935
[0.082]
[0.265 0.087]
0.102


In [11]:
np.random.seed(1234)
def data_gen_2(n = 1000, scale = 1, shape = 2):
    x = np.random.exponential(scale = 1, size = n)
    y = np.random.weibull(a = shape, size = n)
    return np.column_stack((x, y))

scale = 1
shape = 1.5
true = (1 + np.log(scale)) - (-sp.digamma(1)*(1 - 1/shape) + np.log(scale/shape) + 1) 
niter = 200
count = np.zeros(niter)
bias = np.zeros(niter)
estim = np.zeros(niter)
sd_estim = np.zeros(niter)
for i in range(niter):
    data = data_gen_2(500, scale, shape)
    op = estimator(data)
    estim[i] = op[3]
    sd_estim[i] = op[5]
    if((true - op[2])*(op[4] - true) >= 0):
        count[i] = 1
    bias[i] = np.abs(op[3] - true)

In [12]:
print(np.round(np.mean(count), 3)) ## coverage
print(np.round([np.mean(bias)], 3)) ## absolute error and error variance
print(np.round([np.mean(estim), np.std(estim)], 3))## MC-based estimator mean and variance
print(np.round(np.mean(sd_estim), 3))

0.935
[0.062]
[0.257 0.064]
0.074


In [13]:
np.random.seed(1234)
def data_gen_2(n = 1000, scale = 1, shape = 2):
    x = np.random.exponential(scale = 1, size = n)
    y = np.random.weibull(a = shape, size = n)
    return np.column_stack((x, y))

scale = 1
shape = 1.5
true = (1 + np.log(scale)) - (-sp.digamma(1)*(1 - 1/shape) + np.log(scale/shape) + 1) 
niter = 200
count = np.zeros(niter)
bias = np.zeros(niter)
estim = np.zeros(niter)
sd_estim = np.zeros(niter)
for i in range(niter):
    data = data_gen_2(750, scale, shape)
    op = estimator(data)
    estim[i] = op[3]
    sd_estim[i] = op[5]
    if((true - op[2])*(op[4] - true) >= 0):
        count[i] = 1
    bias[i] = np.abs(op[3] - true)

In [14]:
print(np.round(np.mean(count), 3)) ## coverage
print(np.round([np.mean(bias)], 3)) ## absolute error and error variance
print(np.round([np.mean(estim), np.std(estim)], 3))## MC-based estimator mean and variance
print(np.round(np.mean(sd_estim), 3))

0.96
[0.049]
[0.245 0.051]
0.061
